# Практическое задание 4. Тематическое моделирование. EM-алгоритм

## Задание на тематическое моделирование

продолжаем исследование датасета с твитами

**Скачиваем датасет (источник): положительные, отрицательные.**

1. рабочие ссылки с твитами
[https://disk.yandex.ru/i/v5HM-ENiGXZVpQ]()
[https://disk.yandex.ru/i/koR5eMCToCZS2Q]()

2. как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

**Обработка**
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

## Скачивание набора данных

In [8]:
import pandas as pd

df1 = pd.read_csv('positive.csv', delimiter=';', header=None, usecols=[3], names=['text'])
df2 = pd.read_csv('negative.csv', delimiter=';', header=None, usecols=[3], names=['text'])
df = pd.concat([df1, df2], axis=0)

In [9]:
df.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 0 to 111922
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
dtypes: object(1)
memory usage: 3.5+ MB


## Предобработка

In [11]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k142\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Настройка

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
words_regex = re.compile('\w+')
stopwords_list = stopwords.words('russian')

In [14]:
# убираем слова < 3
def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

# лемматизация с проверкой на стоп слова
def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords
            and w.isalpha()]

# объединяем обработку
def preprocess(text):
    return (lemmatize(find_words(text)))

### Проверка обработки

In [15]:
df.text.iloc[1]

'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'

In [16]:
preprocess(df.text.iloc[1])

['всё', 'таки', 'немного', 'похожий', 'мальчик', 'всё', 'равно', 'хороший']

### Обработка корпуса данных

In [17]:
df['document'] = df['text'].apply(lambda x: preprocess(x))

In [18]:
df.head()

,text,document
0,"@first_timee хоть я и школота, но поверь, у на...","[школотый, поверь, самый, общество, профилиров..."
1,"Да, все-таки он немного похож на него. Но мой ...","[всё, таки, немного, похожий, мальчик, всё, ра..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,"[katiacheh, идиотка, испугаться]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...","[угол, сидеть, погибать, голод, ещё, порция, в..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,"[значит, страшилка, блин, посмотреть, всё, час..."


> Оставим 10000 для ускорения

In [20]:
# df_s = df.sample(10000)
df_s = df

In [21]:
df_s.to_pickle('tweet.pkl')

## Подготовка данных

In [22]:
from gensim.models import *
from gensim import corpora

> Создадим словарь и проведем начальную фильтрацию на редкие слова

In [23]:
dictionary = corpora.Dictionary(df_s['document'])
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря
dictionary.save('tweet.dict')

> Векторизуем документы:

In [24]:
corpus = [dictionary.doc2bow(text) for text in df_s['document']]
corpora.MmCorpus.serialize('tweet.model', corpus)

## Обучение модели baseline

> Теперь можем обучать модель:

In [53]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=20, chunksize=50, update_every=1, passes=2)

CPU times: total: 4min 34s
Wall time: 4min 35s


## Интерпретация результата

In [54]:
lda.show_topics(num_topics=10, num_words=5, formatted=True)

[(7,
  '0.000*"снегурочка" + 0.000*"фикбук" + 0.000*"обидеть" + 0.000*"побриться" + 0.000*"лавочка"'),
 (16,
  '0.411*"блин" + 0.167*"друг" + 0.143*"посмотреть" + 0.081*"наверное" + 0.062*"значит"'),
 (6,
  '0.292*"любить" + 0.199*"думать" + 0.129*"первый" + 0.104*"найти" + 0.082*"папа"'),
 (12,
  '0.000*"снегурочка" + 0.000*"фикбук" + 0.000*"обидеть" + 0.000*"побриться" + 0.000*"лавочка"'),
 (13,
  '0.387*"хороший" + 0.177*"равно" + 0.107*"таки" + 0.089*"взять" + 0.071*"немного"'),
 (1,
  '0.758*"всё" + 0.159*"сказать" + 0.062*"понять" + 0.000*"большой" + 0.000*"болеть"'),
 (3,
  '0.000*"снегурочка" + 0.000*"фикбук" + 0.000*"обидеть" + 0.000*"побриться" + 0.000*"лавочка"'),
 (11,
  '0.567*"свой" + 0.295*"каждый" + 0.035*"радовать" + 0.031*"факт" + 0.000*"жизнь"'),
 (19,
  '0.733*"это" + 0.185*"человек" + 0.035*"получаться" + 0.028*"слушать" + 0.000*"вообще"'),
 (17,
  '0.388*"ещё" + 0.218*"просто" + 0.208*"почему" + 0.071*"дело" + 0.047*"сильно"')]

### Вывод
> на первый взгляд требуется приличная чистка на слова не передающие смысл: таки, все, это, еще.

### Визуализация данных

In [67]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda, corpus, dictionary,mds='mmds')
pyLDAvis.display(vis_data)

C:\prj\GeekBrainsAI\intro_nlp\venv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


### Метрики

>перплексия

In [69]:
print(lda.log_perplexity(corpus))

-26.29492709742201

-26.294927096496043


In [70]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  3.804155491347106e-12


> средняя когерентность

In [ ]:
coherence_model_lda = CoherenceModel(model=lda, texts=df_s['document'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

### Вывод
![](baseline.jpg)


> 1. Существует много лишних тем, которые сильно сближены для предположения наличия 20 тем. Уменьшим количество до 14
> 2. Существуют ряд токенов, которые высокочастотны и передают мало смысла: http, это, год. Возможно провести удаление этих слов

## Оптимизация модели

In [25]:

%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=13, chunksize=50, update_every=1, passes=2)

CPU times: total: 4min 9s
Wall time: 4min 11s


In [27]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda, corpus, dictionary,mds='mmds')
pyLDAvis.display(vis_data)

C:\prj\GeekBrainsAI\intro_nlp\venv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


После снижения количества тем исчезли пустые темы и можно некоторые интерпретировать:
**Вопрос в чате**
![](about_qwestion.jpg)

**Реплика про жизнь**
![](about_live.jpg)

**побуждение к действию, обсуждение результата**
![](about_motivation.jpg)

## Вывод
> Подготовил данные для анализа тематик.
> Построил базовую модель.
> Выполнил простейшую оптимизацию.
> Провел интерпретацию некоторых тем.
> Общение короткими сообщениями сложно поддается анализу на темы. Возможно рассмотреть построение фильтра на сообщения информативные против простейших.